# MATH 472 Homework 4
## Rachel Singleton
## Due 02/13/2020

## Problem 2.5

There were 46 crude oil spills of at least 1000 barrels from tankers in the US waters during 1974-1999. The website for this book contains the following data: the number of spills in the $i^{th}$ year, $N_i$; the estimated amount of oil shipped through US waters as part of US import/export operations in the $i^{th}$ year, adjusted for spillage in international or foreign waters, $b_{i1}$; and the amount of oil shipped through US waters during domestic shipments in the $i^{th}$ year, $b_{i2}$. The data are adapted. Oil shipment amounts are measured in billions of barrels (Bbbl).

The volume of oil shipped is a measure of exposure to spill risk. Suppose we use the Poisson process assumption given by $N_i|b_{i1}, b_{i2} ~ Poisson(\lambda_i)$, where $\lambda_i = \alpha_1 b_{i1} + \alpha_2 b_{i2}$. The parameters of this model are $\alpha_1$ and $\alpha_2$, which represent the rate of spill occurrence per Bbbl oil shipped during import/export and domestic shipments, respectively.

### (a) Derive the Newton-Raphson update for finding the MLEs of $\alpha_{1}$ and $\alpha_{2}$

### Answer to part (a)

We know our general Poisson distribution is $f(x) = \frac{\lambda^x}{x!}*e^{-\lambda}$. For our problem, $N_i$ will take the place of $x$ and we will rewrite $\lambda_i$ in vector notation. We can rewrite $[\alpha_1 \alpha_2]^T$ as $[\alpha]^T$ which then allows us to write $\lambda = \alpha^Tb$. 

This gives us a likelihood function of $L(\alpha) = \prod_{i=1}^{n} \frac{1}{N_i!}*(\alpha^Tb_i)^{N_i}*e^{-\alpha^Tb_i}$. 

We can take the log of the likelihood to get the log-likelihood function: $l(\alpha) = \log{L(\alpha)} = \sum_{i=1}^{n} \log{[\frac{1}{N_i!}*(\alpha^Tb_i)^{N_i}*e^{-\alpha^Tb_i}]}$. Now we can simplify it to get

$l(\alpha) = \sum_{i=1}^{n} N_i*\log{(\alpha^Tb_i)} - \sum_{i=1}^{n} \log{(N_i!)} - \sum_{i=1}^{n} \alpha^Tb_i$

Now we will take the derivative of our log-likelihood function:

$l^{'}(\alpha) = \sum_{i=1}^{n} N_i*\frac{b_i}{\alpha^Tb_i} - \sum_{i=1}^{n} b_i$

Lastly, we will take the derivative again to get the 2nd derivative of our log-likelihood function:

$l^{''}(\alpha) = \sum_{i=1}^{n} \frac{-N_i}{(\alpha^Tb_i)^2}*(b_i)^2$

Since $b_i$ is a vector, we can't technically square it so we will rewrite it with a transpose:

$l^{''}(\alpha) = \sum_{i=1}^{n} \frac{-N_i}{(\alpha^Tb_i)^2}*(b_ib_i^T)$

To get the Newton-Raphson update equation, we will plug our $\alpha$ vector and our derivatives of log-likelihood into equation (2.33) from the book:

$\alpha^{(t+1)} = \alpha^{(t)} - [\sum_{i=1}^{n} \frac{-N_i}{((\alpha^{(t)})^Tb_i)^2}*(b_ib_i^T)]^{-1}[\sum_{i=1}^{n} N_i*\frac{b_i}{(\alpha^{(t)})^Tb_i} - \sum_{i=1}^{n} b_i]$

### (b) Derive the Fisher scoring update for finding the MLEs of $\alpha_1$ and $\alpha_2$

### Answer to part (b)

To find the Fisher Scoring update, we will use equation (2.34) from the book, equations from part (a), and the definition of the Fisher Information matrix which is $I(\theta) = -E\{l^{''}(\theta)\}$.

We will first find the Fisher Information matrix:

$I(\alpha) = -E[\sum_{i=1}^{n} \frac{-N_i}{(\alpha^Tb_i)^2}*(b_ib_i^T)] = -\sum_{i=1}^{n} E[\frac{-N_i}{(\alpha^Tb_i)^2}*(b_ib_i^T)] = -\sum_{i=1}^{n} \frac{-b_ib_i^T}{(\alpha^Tb_i)^2}*E[N_i]$

From knowing that $N_i$ is from a Poisson distribution, we know that the expected value of $N_i$ is $\lambda_i$ which we know is equal to $\alpha^Tb_i$. This then gives us

$I(\alpha) = -\sum_{i=1}^{n} \frac{-b_ib_i^T}{(\alpha^Tb_i)^2}*(\alpha^Tb_i) = -\sum_{i=1}^{n} \frac{-b_ib_i^T}{(\alpha^Tb_i)} = \sum_{i=1}^{n} \frac{b_ib_i^T}{(\alpha^Tb_i)}$

Now we can plug the Fisher Information matrix into our updating equation:

$\alpha^{(t+1)} = \alpha^{(t)} + [\sum_{i=1}^{n} \frac{b_ib_i^T}{(\alpha^{(t)})^Tb_i}]^{-1}*[\sum_{i=1}^{n} N_i*\frac{b_i}{(\alpha^{(t)})^Tb_i} - \sum_{i=1}^{n} b_i]$

### (c) Implement the Newton-Raphson and Fisher scoring methods for this problem, provide the MLEs of $\alpha_1$ and $\alpha_2$, and compare the implementation ease and performance of the two methods.

In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import inv
from math import sqrt

data = pd.read_csv("oilspills.dat", delimiter=" ")
N = data['spills'].to_numpy().reshape((26,1))
b1 = data['importexport'].to_numpy().reshape((26,1))
b2 = data['domestic'].to_numpy().reshape((26,1))

n = N.size
b = np.hstack((b1,b2)).reshape((26,2))
alpha = np.asarray([0.5,0.5]).reshape((2,1))

appendedList = []

def likelihood(alpha):
    total = []
    for i in range(26):
        b_i = b[i].reshape((2,1))
        total.append(N[i]*np.log(alpha.T.dot(b_i)) - np.log(np.math.factorial(N[i])) - (alpha.T.dot(b_i)))
    total = np.asarray(total)
    summation = total.sum(axis=0)
    return summation

def lprime(alpha):
    total = []
    for i in range(26):
        b_i = b[i].reshape((2,1))
        total.append(((N[i]/((alpha.T).dot(b_i)))*b_i)-b_i)
    total = np.asarray(total)
    summation = total.sum(axis=0)
    return summation

def lprime_2(alpha):
    total = []
    for i in range(n):
        b_i = b[i].reshape((2,1))
        total.append((-N[i]/((alpha.T).dot(b_i)**2))*(b_i.dot(b_i.T)))
    total = np.asarray(total)
    summation = total.sum(axis=0)
    return summation

def fisher(alpha):
    total = []
    for i in range(n):
        b_i = b[i].reshape((2,1))
        total.append((1/(alpha.T.dot(b_i)))*(b_i.dot(b_i.T)))
    total = np.asarray(total)
    summation = total.sum(axis=0)
    return summation

def newton_method(alpha,param):
    alpha_values = [alpha]
    num_iterations = 0
    max_iterations = 7
    data_values = []
    
    while((num_iterations < max_iterations)):
        alpha = alpha - inv(lprime_2(alpha)).dot(lprime(alpha).reshape((2,1)))
        data_values = [num_iterations,alpha]
        alpha_values.append(alpha)
        appendedList.append(data_values)
        num_iterations+=1
    
    # Printing results
    if(param==1):
        if(num_iterations == max_iterations):
            print("You reached the maximum number of iterations.")
        else:
            print("Number of iterations: ",num_iterations)
        print("Final Solution: ", alpha_values)
    elif(param==2):
        return alpha_values

newton_method(alpha,2)
   
print("Table for Starting Alphas [0.5,0.5] Using Newton's Method")
df = pd.DataFrame(appendedList, columns = ['Iteration,t','Alphas'])
df = df.iloc[2:7,:]
df = df.style.hide_index()
df = df.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
df

Table for Starting Alphas [0.5,0.5] Using Newton's Method


"Iteration,t",Alphas
2,[[1.08904959] [0.9364765 ]]
3,[[1.09709646] [0.93756821]]
4,[[1.09715253] [0.93755459]]
5,[[1.09715253] [0.93755458]]
6,[[1.09715253] [0.93755458]]


In [2]:
appendedList1 = []
fisher_error = []
def fisher_method(alpha,param):
    alpha_values = [alpha]
    num_iterations = 0
    max_iterations = 15
    data_values1 = []
    
    while((num_iterations < max_iterations)):
        alpha = alpha + inv(fisher(alpha)).dot(lprime(alpha).reshape((2,1)))
        fisher_error.append(inv(fisher(alpha)))
        data_values1 = [num_iterations,alpha]
        alpha_values.append(alpha)
        appendedList1.append(data_values1)
        num_iterations+=1
    
    # Printing results
    if(param==1):
        if(num_iterations == max_iterations):
            print("You reached the maximum number of iterations.")
        else:
            print("Number of iterations: ",num_iterations)
        print("Final Solution: ", alpha_values)
    elif(param==2):
        return alpha_values
    
fisher_method(alpha,2)
print("Table for Starting Alphas [0.5,0.5] Using Fisher Scoring")
df = pd.DataFrame(appendedList1, columns = ['Iteration,t','Alphas'])
df = df.iloc[10:15,:]
df = df.style.hide_index()
df = df.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
df

Table for Starting Alphas [0.5,0.5] Using Fisher Scoring


"Iteration,t",Alphas
10,[[1.09715273] [0.93755428]]
11,[[1.09715247] [0.93755468]]
12,[[1.09715255] [0.93755455]]
13,[[1.09715253] [0.93755459]]
14,[[1.09715253] [0.93755458]]


### Answer to part (c)

Between Newton's Method and Fisher Scoring, Newton's was more efficient because it converged in 4 iterations while Fisher converged in 13 iterations. This means that Newton's outperforms Fisher and reduces the amount of time you would have to run the algorithm to find your answer. I thought that Fisher Scoring was easier to implement but that was because I had already gone through the Newton implementation and they are practically the same. In terms of "first glance" implementation, I would say that Newton's method is easier to implement because you don't have to do the expected value of the Hessian like you do with Fisher. 

### (d) Estimate standard errors for the MLEs of $\alpha_1$ and $\alpha_2$

In [3]:
diags = []
values = []
list = []
for i in range(len(fisher_error)):
    val = np.diag(fisher_error[i])
    val = [sqrt(val[0]),sqrt(val[1])]
    diags.append(val)
    values = [i,val]
    list.append(values)
    
print("Table or Errors for Alphas [0.5,0.5]")
df = pd.DataFrame(list, columns = ['Iteration,t','Error of Alpha'])
df = df.iloc[10:15,:]
df = df.style.hide_index()
df = df.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
df

Table or Errors for Alphas [0.5,0.5]


"Iteration,t",Error of Alpha
10,"[0.43755598263643175, 0.6314686971816681]"
11,"[0.43755596869791896, 0.6314687109144207]"
12,"[0.43755597309782224, 0.6314687065794696]"
13,"[0.43755597170892563, 0.6314687079478624]"
14,"[0.43755597214735215, 0.6314687075159088]"


### (e) Apply the method of steepest ascent. Use step-halving backtracking as necessary.

In [4]:
def ascent_method(init_alpha):
    step = 1
    num_iterations = 0
    max_iterations = 75
    
    alpha_new = alpha + step*lprime(init_alpha)
    alpha_list = np.hstack([init_alpha,alpha_new])
    alpha_old = init_alpha
    check1 = alpha_new[0]-alpha_old[0]
    check2 = alpha_new[1]-alpha_old[1]
    check = np.linalg.norm([check1,check2],axis=0)
    while(num_iterations < max_iterations) & (check > 1e-6):
        alpha_old = np.array([alpha_list[:, -1]]).T
        alpha_new = alpha_old + step*lprime(alpha_old)
        g_old = likelihood(alpha_old)
        g_new = likelihood(alpha_new)
        if((g_new > g_old) & (np.isfinite(g_new))):
            alpha_list = np.hstack((alpha_list, alpha_new))
            num_iterations += 1
            check1 = alpha_new[0]-alpha_old[0]
            check2 = alpha_new[1]-alpha_old[1]
            check = np.linalg.norm([check1,check2],axis=0)
        else:
            step = step/2
    
    print(num_iterations)
    print(alpha_list)

ascent_method(alpha)

73
[[ 0.5        29.5356457   3.80924846  1.56241106  1.34665605  1.35237349
   1.29353832  1.27080593  1.24089855  1.22004772  1.20068336  1.18504021
   1.17153215  1.16022461  1.15061042  1.14249037  1.13560577  1.12977581
   1.1248334   1.12064371  1.11709052  1.11407671  1.11151978  1.10935016
   1.10750888  1.10594607  1.10461947  1.10349328  1.10253713  1.10172531
   1.10103599  1.10045066  1.0999536   1.09953149  1.09917302  1.09886859
   1.09861004  1.09839046  1.09820396  1.09804557  1.09791104  1.09779678
   1.09769973  1.09761731  1.0975473   1.09748784  1.09743733  1.09739443
   1.097358    1.09732705  1.09730076  1.09727844  1.09725947  1.09724337
   1.09722968  1.09721806  1.09720819  1.09719981  1.09719269  1.09718664
   1.0971815   1.09717714  1.09717343  1.09717029  1.09716761  1.09716534
   1.09716341  1.09716177  1.09716038  1.0971592   1.0971582   1.09715734
   1.09715662  1.097156    1.09715548]
 [ 0.5        18.9643543   2.01232263  0.56785722  0.50841594  0.60418

/Users/Rachel/VIP_Stuff/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in log


### (f) Apply quasi-Newton optimization with the Hessian approximation update given in (2.49). Compare performance with and without step-halving.

### Answer to part (f)

### (g) Construct a graph resemling Figure 2.8 that compares the paths taken by method used in (a)-(f). Choose the plotting region and starting point to best illustrate the features of the algorithms' performance.